In [49]:
import s3fs

import numpy as np
import torch
import xarray as xr

import cartopy.crs as ccrs

import plot


In [6]:
fs = s3fs.S3FileSystem(anon=True)
url = f's3://hrrrzarr/grid/HRRR_chunk_index.zarr'
file = s3fs.S3Map(url, s3=fs)

In [19]:
# get location data and drop other variables
locations = xr.open_dataset(file, engine='zarr')
locations = locations.drop_vars(list(locations.data_vars.keys()))

In [20]:
# add grid coordinates
X,Y = np.meshgrid(locations.x, locations.y, indexing='xy')
locations = locations.assign_coords(
    X=(['y', 'x'], X),
    Y=(['y', 'x'], Y)
)

In [24]:
# get DPT data
fs = s3fs.S3FileSystem(anon=True)
url =  f's3://hrrrzarr/sfc/20210215/20210215_01z_fcst.zarr/500mb/DPT/500mb'
file = s3fs.S3Map(url, s3=fs)

In [42]:
# renmae dimensions for convenience
obs = xr.open_dataset(file, engine='zarr').load()
obs = obs.rename({
    'projection_x_coordinate': 'x',
    'projection_y_coordinate': 'y'
})
obs = obs.rename_vars({
    'DPT': 'obs'})

In [44]:
# merge
data = xr.merge([obs, locations])

In [45]:
# 

Lambert_proj = ccrs.LambertConformal(central_longitude=262.5, central_latitude=38.5,
                                standard_parallels=[38.5,38.5],
                                globe=ccrs.Globe(semimajor_axis=6371229, 
                                                 semiminor_axis=6371229))

In [ ]:
plot.obs(data, Lambert_proj)

In [70]:
data_sub = data.isel(y=slice(None, None, 55), x=slice(None, None, 45))

In [66]:
n = data_sub.obs.shape[0]-1
targets = torch.tensor(data_sub.obs.values,dtype=torch.float32).reshape(n+1,-1)/50
inputs = torch.tensor(np.stack([data_sub.X,data_sub.Y], axis = -1).reshape(-1,2), dtype=torch.float32)/1e6